# ACS Data Cleaning - Top 10 Metro Areas

## Importing Required Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import pandas_gbq
import statistics
from google.cloud import bigquery
from varname import nameof
from collections import Counter
%load_ext google.cloud.bigquery

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '../zori-data-extr-be793d5c3325.json'

# Set your default project here
pandas_gbq.context.project = 'bigquery-public-data'
pandas_gbq.context.dialect = 'standard'

## Using Google BigQuery to Download ACS Data

In [2]:
%%bigquery --use_rest_api ACS_2018
SELECT *
FROM `bigquery-public-data.census_bureau_acs.zip_codes_2018_5yr`

Downloading: 100%|██████████| 33120/33120 [00:05<00:00, 6068.51rows/s] 


In [3]:
%%bigquery --use_rest_api ACS_2017
SELECT *
FROM `bigquery-public-data.census_bureau_acs.zip_codes_2017_5yr`

Downloading: 100%|██████████| 33120/33120 [00:05<00:00, 5890.60rows/s] 


In [4]:
%%bigquery --use_rest_api ACS_2016
SELECT *
FROM `bigquery-public-data.census_bureau_acs.zip_codes_2016_5yr`

Downloading: 100%|██████████| 33120/33120 [00:05<00:00, 5684.05rows/s]


In [5]:
%%bigquery --use_rest_api ACS_2015
SELECT *
FROM `bigquery-public-data.census_bureau_acs.zip_codes_2015_5yr`

Downloading: 100%|██████████| 33120/33120 [00:14<00:00, 2260.07rows/s]


In [6]:
%%bigquery --use_rest_api ACS_2014
SELECT *
FROM `bigquery-public-data.census_bureau_acs.zip_codes_2014_5yr`

Downloading: 100%|██████████| 33120/33120 [00:06<00:00, 5161.93rows/s]


## ACS Data Pre-Processing

In [7]:
pd.set_option('display.max_columns', None)
#Looking at the shape of each dataframe
print(ACS_2018.shape)
print(ACS_2017.shape)
print(ACS_2016.shape)
print(ACS_2015.shape)
print(ACS_2014.shape, '\n')

(33120, 240)
(33120, 252)
(33120, 252)
(33120, 247)
(33120, 252) 



Based on the data shown above, a number of columns are missing from the 2018 data set. Since the 2018 data set is the latest, all additional columns seen in prior year surveys will be dropped for consistency.

In [8]:
def clean_columns(df_year1, df_year2):
    """
    This function cleans the second dataframe to only include columns of the first dataframe.
    
    Arguments
    
    df_year1 : (Pandas dataframe) dataframe containing the columns of interest
    df_year2 : (Pandas dataframe) dataframe containing the columns of interest + additional
    
    Returns
    
    df_year2 : (Pandas dataframe) modified version of the df_year2 dataframe from inputs
    
    """
    diff = np.setdiff1d(df_year2.columns, df_year1.columns)
    print(f'Columns from {nameof(df_year2)} non included in {nameof(df_year1)}:\n\nTotal of {len(diff)}\n\n{diff}\n\n')
    print('Removing columns...')
    
    #Dropping the columns from df_year2 that are not present in the df_year1
    df_year2 = df_year2.drop(columns = diff, inplace = True)
    
    print('Process complete\n\n')
    
    return df_year2

In [9]:
#Using the above defined function to clean columns
clean_columns(ACS_2018, ACS_2017)
clean_columns(ACS_2018, ACS_2016)
clean_columns(ACS_2018, ACS_2015)
clean_columns(ACS_2018, ACS_2014)

Columns from df_year2 non included in df_year1:

Total of 12

['amerindian_including_hispanic' 'asian_including_hispanic'
 'black_including_hispanic' 'commute_35_39_mins' 'commute_40_44_mins'
 'commute_5_9_mins' 'commute_60_89_mins' 'commute_90_more_mins'
 'households_retirement_income' 'male_60_61' 'male_62_64'
 'white_including_hispanic']


Removing columns...
Process complete


Columns from df_year2 non included in df_year1:

Total of 12

['amerindian_including_hispanic' 'asian_including_hispanic'
 'black_including_hispanic' 'commute_35_39_mins' 'commute_40_44_mins'
 'commute_5_9_mins' 'commute_60_89_mins' 'commute_90_more_mins'
 'households_retirement_income' 'male_60_61' 'male_62_64'
 'white_including_hispanic']


Removing columns...
Process complete


Columns from df_year2 non included in df_year1:

Total of 13

['amerindian_including_hispanic' 'asian_including_hispanic'
 'black_including_hispanic' 'commute_35_39_mins' 'commute_40_44_mins'
 'commute_5_9_mins' 'commute_60_89_mins'

In [10]:
#Looking at the shape of each dataframe to confirm column cleaning has been done appropriately
print(ACS_2018.shape)
print(ACS_2017.shape)
print(ACS_2016.shape)
print(ACS_2015.shape)
print(ACS_2014.shape, '\n')

(33120, 240)
(33120, 240)
(33120, 240)
(33120, 234)
(33120, 240) 



In [11]:
diff = np.setdiff1d(ACS_2018.columns, ACS_2015.columns)
diff

array(['pop_15_and_over', 'pop_divorced', 'pop_never_married',
       'pop_now_married', 'pop_separated', 'pop_widowed'], dtype=object)

As can be seen above for year 2015 there are less columns than in any other year. Before dropping the columns in all other survey years, looking at missingness of those 6 columns would be best.

In [12]:
#Adding a year column to each survey dataframe
ACS_2018['year'] = 2018
ACS_2017['year'] = 2017
ACS_2016['year'] = 2016
ACS_2015['year'] = 2015
ACS_2014['year'] = 2014

#Combining all ACS dataframes to one global ACS dataframe
ACS = pd.concat([ACS_2018, ACS_2017, ACS_2016, ACS_2015, ACS_2014], ignore_index = True)
ACS.reset_index().drop(columns = 'index', inplace = True)

In [13]:
ACS.shape

(165600, 241)

### Keeping the Focus on Top 10 Zillow Metro Areas

The project is looking at predicting 2019 data as a base model (no Covid-19 effect) by using zip codes from the top 10 metro areas in the currently available ZRI index data. 

In [14]:
#Reading in the Focus Zipcode file
focus_zip = pd.read_csv('../Data/focus_zipcode.csv')

#Creating a list of pertinent zip codes in str format
focus_zip_l = focus_zip.RegionName.apply(lambda x: str(x)).to_list()

#Keeping rows that have focus zipcodes in created ACS dataframes
ACS = ACS.loc[ACS['geo_id'].isin(focus_zip_l)]

In [15]:
ACS.shape

(11165, 241)

### ACS Operational Data

Columns such as *gini_index* and *do_date* are ACS operational columns that are not required for this project. As such, they will be dropped.

In [16]:
#Dropping ACS operational columns
ACS.drop(columns = ['gini_index', 'do_date'], inplace = True)

### Income Columns

Partly Parrots will be looking at IRS data for everything income related, as such all income related columns will be dropped.

In [17]:
ACS = ACS[ACS.columns.drop(list(ACS.filter(regex = 'income')))]

In [18]:
ACS.shape

(11165, 220)

### Looking at Missingness

In [19]:
def missingness(df):
    """
    This function looks at the number of missing values in a dataframe
    
    Arguments
    
    df          : (Pandas dataframe) dataframe of interest
    
    Returns
    
    missing_col : (dictionary) column name and number of missing values 
    
    """
    all_cols_miss = df.isna().mean().sort_values(ascending = False)

    missing_col_list = []
    missing_col = {}

    for i in range(all_cols_miss[all_cols_miss > 0].shape[0]):
        missing_col_list.append([all_cols_miss[all_cols_miss > 0].index[i], \
                                round(all_cols_miss[all_cols_miss > 0][i]*100, 2)])

    missing_col.update(missing_col_list)
    return missing_col

In [20]:
#Looking at missing values in data
missingness(ACS)

{'pop_now_married': 80.0,
 'pop_15_and_over': 80.0,
 'pop_divorced': 80.0,
 'pop_widowed': 80.0,
 'pop_separated': 80.0,
 'pop_never_married': 80.0,
 'pop_5_years_over': 60.0,
 'speak_spanish_at_home': 60.0,
 'speak_only_english_at_home': 60.0,
 'speak_spanish_at_home_low_english': 60.0,
 'aggregate_travel_time_to_work': 1.36,
 'median_rent': 0.46,
 'renter_occupied_housing_units_paying_cash_median_gross_rent': 0.4,
 'owner_occupied_housing_units_lower_value_quartile': 0.14,
 'median_year_structure_built': 0.1,
 'owner_occupied_housing_units_median_value': 0.07,
 'owner_occupied_housing_units_upper_value_quartile': 0.06}

##### Taking a closer look into low missingness rows<br>
##### Aggregate Travel Time to Work

*Typically during census data collection, questions might change from year-to-year, leading to missingness in data througohut a longer time period (years). As such, checking missingness by year is crucial to have a better understanding of the data collection process. As the percentage for missing data in aggregate travel time to work is small, this check will not be required (less than 10% of a years worth of questions).*

In [21]:
ACS.loc[ACS['aggregate_travel_time_to_work'].isna()].head(3)

,geo_id,total_pop,households,male_pop,female_pop,median_age,male_under_5,male_5_to_9,male_10_to_14,male_15_to_17,male_18_to_19,male_20,male_21,male_22_to_24,male_25_to_29,male_30_to_34,male_35_to_39,male_40_to_44,male_45_to_49,male_50_to_54,male_55_to_59,male_65_to_66,male_67_to_69,male_70_to_74,male_75_to_79,male_80_to_84,male_85_and_over,female_under_5,female_5_to_9,female_10_to_14,female_15_to_17,female_18_to_19,female_20,female_21,female_22_to_24,female_25_to_29,female_30_to_34,female_35_to_39,female_40_to_44,female_45_to_49,female_50_to_54,female_55_to_59,female_60_to_61,female_62_to_64,female_65_to_66,female_67_to_69,female_70_to_74,female_75_to_79,female_80_to_84,female_85_and_over,white_pop,population_1_year_and_over,population_3_years_over,pop_5_years_over,pop_15_and_over,pop_16_over,pop_25_years_over,pop_25_64,pop_never_married,pop_now_married,pop_separated,pop_widowed,pop_divorced,not_us_citizen_pop,black_pop,asian_pop,hispanic_pop,amerindian_pop,other_race_pop,two_or_more_races_pop,hispanic_any_race,not_hispanic_pop,asian_male_45_54,asian_male_55_64,black_male_45_54,black_male_55_64,hispanic_male_45_54,hispanic_male_55_64,white_male_45_54,white_male_55_64,pop_determined_poverty_status,poverty,housing_units,renter_occupied_housing_units_paying_cash_median_gross_rent,owner_occupied_housing_units_lower_value_quartile,owner_occupied_housing_units_median_value,owner_occupied_housing_units_upper_value_quartile,occupied_housing_units,housing_units_renter_occupied,vacant_housing_units,vacant_housing_units_for_rent,vacant_housing_units_for_sale,dwellings_1_units_detached,dwellings_1_units_attached,dwellings_2_units,dwellings_3_to_4_units,dwellings_5_to_9_units,dwellings_10_to_19_units,dwellings_20_to_49_units,dwellings_50_or_more_units,mobile_homes,housing_built_2005_or_later,housing_built_2000_to_2004,housing_built_1939_or_earlier,median_year_structure_built,married_households,nonfamily_households,family_households,households_public_asst_or_food_stamps,male_male_households,female_female_households,children,children_in_single_female_hh,median_rent,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,rent_35_to_40_percent,rent_30_to_35_percent,rent_25_to_30_percent,rent_20_to_25_percent,rent_15_to_20_percent,rent_10_to_15_percent,rent_under_10_percent,owner_occupied_housing_units,million_dollar_housing_units,mortgaged_housing_units,different_house_year_ago_different_city,different_house_year_ago_same_city,families_with_young_children,two_parent_families_with_young_children,two_parents_in_labor_force_families_with_young_children,two_parents_father_in_labor_force_families_with_young_children,two_parents_mother_in_labor_force_families_with_young_children,two_parents_not_in_labor_force_families_with_young_children,one_parent_families_with_young_children,father_one_parent_families_with_young_children,father_in_labor_force_one_parent_families_with_young_children,commute_less_10_mins,commute_10_14_mins,commute_15_19_mins,commute_20_24_mins,commute_25_29_mins,commute_30_34_mins,commute_35_44_mins,commute_60_more_mins,commute_45_59_mins,commuters_16_over,walked_to_work,worked_at_home,no_car,no_cars,one_car,two_cars,three_cars,four_more_cars,aggregate_travel_time_to_work,commuters_by_public_transportation,commuters_by_bus,commuters_by_car_truck_van,commuters_by_carpool,commuters_by_subway_or_elevated,commuters_drove_alone,group_quarters,associates_degree,bachelors_degree,high_school_diploma,less_one_year_college,masters_degree,one_year_more_college,less_than_high_school_graduate,high_school_including_ged,bachelors_degree_2,bachelors_degree_or_higher_25_64,graduate_professional_degree,some_college_and_associates_degree,male_45_64_associates_degree,male_45_64_bachelors_degree,male_45_64_graduate_degree,male_45_64_less_than_9_grade,male_45_64_grade_9_12,male_45_64_high_school,male_45_64_some_college,male_45_to_64,employed_pop,unemployed_pop,pop_in_labor_force,not_in_labor_force,workers_16_and_over,armed_forces,civilian_lab

In [22]:
ACS.loc[~ACS['aggregate_travel_time_to_work'].isna()].head(3)

,geo_id,total_pop,households,male_pop,female_pop,median_age,male_under_5,male_5_to_9,male_10_to_14,male_15_to_17,male_18_to_19,male_20,male_21,male_22_to_24,male_25_to_29,male_30_to_34,male_35_to_39,male_40_to_44,male_45_to_49,male_50_to_54,male_55_to_59,male_65_to_66,male_67_to_69,male_70_to_74,male_75_to_79,male_80_to_84,male_85_and_over,female_under_5,female_5_to_9,female_10_to_14,female_15_to_17,female_18_to_19,female_20,female_21,female_22_to_24,female_25_to_29,female_30_to_34,female_35_to_39,female_40_to_44,female_45_to_49,female_50_to_54,female_55_to_59,female_60_to_61,female_62_to_64,female_65_to_66,female_67_to_69,female_70_to_74,female_75_to_79,female_80_to_84,female_85_and_over,white_pop,population_1_year_and_over,population_3_years_over,pop_5_years_over,pop_15_and_over,pop_16_over,pop_25_years_over,pop_25_64,pop_never_married,pop_now_married,pop_separated,pop_widowed,pop_divorced,not_us_citizen_pop,black_pop,asian_pop,hispanic_pop,amerindian_pop,other_race_pop,two_or_more_races_pop,hispanic_any_race,not_hispanic_pop,asian_male_45_54,asian_male_55_64,black_male_45_54,black_male_55_64,hispanic_male_45_54,hispanic_male_55_64,white_male_45_54,white_male_55_64,pop_determined_poverty_status,poverty,housing_units,renter_occupied_housing_units_paying_cash_median_gross_rent,owner_occupied_housing_units_lower_value_quartile,owner_occupied_housing_units_median_value,owner_occupied_housing_units_upper_value_quartile,occupied_housing_units,housing_units_renter_occupied,vacant_housing_units,vacant_housing_units_for_rent,vacant_housing_units_for_sale,dwellings_1_units_detached,dwellings_1_units_attached,dwellings_2_units,dwellings_3_to_4_units,dwellings_5_to_9_units,dwellings_10_to_19_units,dwellings_20_to_49_units,dwellings_50_or_more_units,mobile_homes,housing_built_2005_or_later,housing_built_2000_to_2004,housing_built_1939_or_earlier,median_year_structure_built,married_households,nonfamily_households,family_households,households_public_asst_or_food_stamps,male_male_households,female_female_households,children,children_in_single_female_hh,median_rent,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,rent_35_to_40_percent,rent_30_to_35_percent,rent_25_to_30_percent,rent_20_to_25_percent,rent_15_to_20_percent,rent_10_to_15_percent,rent_under_10_percent,owner_occupied_housing_units,million_dollar_housing_units,mortgaged_housing_units,different_house_year_ago_different_city,different_house_year_ago_same_city,families_with_young_children,two_parent_families_with_young_children,two_parents_in_labor_force_families_with_young_children,two_parents_father_in_labor_force_families_with_young_children,two_parents_mother_in_labor_force_families_with_young_children,two_parents_not_in_labor_force_families_with_young_children,one_parent_families_with_young_children,father_one_parent_families_with_young_children,father_in_labor_force_one_parent_families_with_young_children,commute_less_10_mins,commute_10_14_mins,commute_15_19_mins,commute_20_24_mins,commute_25_29_mins,commute_30_34_mins,commute_35_44_mins,commute_60_more_mins,commute_45_59_mins,commuters_16_over,walked_to_work,worked_at_home,no_car,no_cars,one_car,two_cars,three_cars,four_more_cars,aggregate_travel_time_to_work,commuters_by_public_transportation,commuters_by_bus,commuters_by_car_truck_van,commuters_by_carpool,commuters_by_subway_or_elevated,commuters_drove_alone,group_quarters,associates_degree,bachelors_degree,high_school_diploma,less_one_year_college,masters_degree,one_year_more_college,less_than_high_school_graduate,high_school_including_ged,bachelors_degree_2,bachelors_degree_or_higher_25_64,graduate_professional_degree,some_college_and_associates_degree,male_45_64_associates_degree,male_45_64_bachelors_degree,male_45_64_graduate_degree,male_45_64_less_than_9_grade,male_45_64_grade_9_12,male_45_64_high_school,male_45_64_some_college,male_45_to_64,employed_pop,unemployed_pop,pop_in_labor_force,not_in_labor_force,workers_16_and_over,armed_forces,civilian_lab

Looking at how the aggregate time to work is calculated, the missing values in this column can be imputed by multiplying the number of commuters in a certain time frame with the average commute time.

In [23]:
commute_l = ACS.filter(regex = 'commute_').columns.to_list()

#Extracting time segments from column titles
commute_times = list(map(lambda x: x.split(sep = '_'), commute_l))

#Calculating average commute times for the time segments
commute_times = list(map(lambda y: statistics.mean(y), 
                         list(map(lambda x: [int(word) for word in x if word.isdigit()], commute_times))))

print(f'Average commute times: {commute_times}')

#Creating a dataframe focusing on missing data in aggregate travel time to work
commute_df = ACS.loc[ACS['aggregate_travel_time_to_work'].isna()][commute_l].astype(float)

#Only looking at commute time columns
commute_df.iloc[:, :] *= commute_times

#Calculating the aggregate time to work
commute_df['aggregate_travel_time_to_work'] = commute_df.sum(axis = 1)

#Dropping all columns but aggregate time to work
commute_df.drop(columns = commute_l, inplace = True)

#Replacing missing values with imputed values
ACS.loc[ACS['aggregate_travel_time_to_work'].isna(), 'aggregate_travel_time_to_work'] = commute_df

missing_rows = ACS.loc[ACS['aggregate_travel_time_to_work'].isna()].shape[0]

#Checking for missingness
print(f'Rows with missingness in agg travel time to work: {missing_rows}')

#Resetting index of dataframe
ACS.reset_index(inplace = True)
ACS.drop(columns = 'index', inplace = True)

Average commute times: [10, 12, 17, 22, 27, 32, 39.5, 60, 52]
Rows with missingness in agg travel time to work: 0


##### Median Rent

*Checking in what year missingness occurred will not be required for this portion as the percentage of missingness is very small.*

In [24]:
ACS.loc[ACS['median_rent'].isna()].head(3)

,geo_id,total_pop,households,male_pop,female_pop,median_age,male_under_5,male_5_to_9,male_10_to_14,male_15_to_17,male_18_to_19,male_20,male_21,male_22_to_24,male_25_to_29,male_30_to_34,male_35_to_39,male_40_to_44,male_45_to_49,male_50_to_54,male_55_to_59,male_65_to_66,male_67_to_69,male_70_to_74,male_75_to_79,male_80_to_84,male_85_and_over,female_under_5,female_5_to_9,female_10_to_14,female_15_to_17,female_18_to_19,female_20,female_21,female_22_to_24,female_25_to_29,female_30_to_34,female_35_to_39,female_40_to_44,female_45_to_49,female_50_to_54,female_55_to_59,female_60_to_61,female_62_to_64,female_65_to_66,female_67_to_69,female_70_to_74,female_75_to_79,female_80_to_84,female_85_and_over,white_pop,population_1_year_and_over,population_3_years_over,pop_5_years_over,pop_15_and_over,pop_16_over,pop_25_years_over,pop_25_64,pop_never_married,pop_now_married,pop_separated,pop_widowed,pop_divorced,not_us_citizen_pop,black_pop,asian_pop,hispanic_pop,amerindian_pop,other_race_pop,two_or_more_races_pop,hispanic_any_race,not_hispanic_pop,asian_male_45_54,asian_male_55_64,black_male_45_54,black_male_55_64,hispanic_male_45_54,hispanic_male_55_64,white_male_45_54,white_male_55_64,pop_determined_poverty_status,poverty,housing_units,renter_occupied_housing_units_paying_cash_median_gross_rent,owner_occupied_housing_units_lower_value_quartile,owner_occupied_housing_units_median_value,owner_occupied_housing_units_upper_value_quartile,occupied_housing_units,housing_units_renter_occupied,vacant_housing_units,vacant_housing_units_for_rent,vacant_housing_units_for_sale,dwellings_1_units_detached,dwellings_1_units_attached,dwellings_2_units,dwellings_3_to_4_units,dwellings_5_to_9_units,dwellings_10_to_19_units,dwellings_20_to_49_units,dwellings_50_or_more_units,mobile_homes,housing_built_2005_or_later,housing_built_2000_to_2004,housing_built_1939_or_earlier,median_year_structure_built,married_households,nonfamily_households,family_households,households_public_asst_or_food_stamps,male_male_households,female_female_households,children,children_in_single_female_hh,median_rent,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,rent_35_to_40_percent,rent_30_to_35_percent,rent_25_to_30_percent,rent_20_to_25_percent,rent_15_to_20_percent,rent_10_to_15_percent,rent_under_10_percent,owner_occupied_housing_units,million_dollar_housing_units,mortgaged_housing_units,different_house_year_ago_different_city,different_house_year_ago_same_city,families_with_young_children,two_parent_families_with_young_children,two_parents_in_labor_force_families_with_young_children,two_parents_father_in_labor_force_families_with_young_children,two_parents_mother_in_labor_force_families_with_young_children,two_parents_not_in_labor_force_families_with_young_children,one_parent_families_with_young_children,father_one_parent_families_with_young_children,father_in_labor_force_one_parent_families_with_young_children,commute_less_10_mins,commute_10_14_mins,commute_15_19_mins,commute_20_24_mins,commute_25_29_mins,commute_30_34_mins,commute_35_44_mins,commute_60_more_mins,commute_45_59_mins,commuters_16_over,walked_to_work,worked_at_home,no_car,no_cars,one_car,two_cars,three_cars,four_more_cars,aggregate_travel_time_to_work,commuters_by_public_transportation,commuters_by_bus,commuters_by_car_truck_van,commuters_by_carpool,commuters_by_subway_or_elevated,commuters_drove_alone,group_quarters,associates_degree,bachelors_degree,high_school_diploma,less_one_year_college,masters_degree,one_year_more_college,less_than_high_school_graduate,high_school_including_ged,bachelors_degree_2,bachelors_degree_or_higher_25_64,graduate_professional_degree,some_college_and_associates_degree,male_45_64_associates_degree,male_45_64_bachelors_degree,male_45_64_graduate_degree,male_45_64_less_than_9_grade,male_45_64_grade_9_12,male_45_64_high_school,male_45_64_some_college,male_45_to_64,employed_pop,unemployed_pop,pop_in_labor_force,not_in_labor_force,workers_16_and_over,armed_forces,civilian_lab

With no way to calculate the median rent, these rows will be dropped.

In [25]:
def clean_missingness_rows(col_names, df):
    """
    This function drops rows with NaN values from a specific list of columns
    
    Arguments
    
    col_names : (list) list of columns with riws containing NaN values
    df        : (Pandas dataframe) dataframe of interest
    
    Returns
    
    df        : (Pandas dataframe) cleaned version of inputted dataframe
    
    """
    df.dropna(axis = 0, subset = col_names, inplace = True)
    for col in df[col_names]:
        missing_rows = col.count('Nan')
        if missing_rows > 0:
            raise ValueError('Cleaning incomplete!')
        else:
            print(f'Rows with missingness in {col}: {missing_rows}\nCleaning complete.\n\n')

In [26]:
clean_missingness_rows(['median_rent'], ACS)

Rows with missingness in median_rent: 0
Cleaning complete.




##### Other

*Checking in what year missingness occurred will not be required for this portion as the percentage of missingness is very small.*

For columns *renter_occupied_housing_units_paying_cash_median_gross_rent*, *owner_occupied_housing_units_lower_value_quartile*, *median_year_structure_built*, *owner_occupied_housing_units_median_value*, and *owner_occupied_housing_units_upper_value_quartile* it will not be feasible to impute these values as they are single value and not derived from other columns based on Partly Parrots's analysis. As such, these rows will be dropped.

In [27]:
clean_missingness_rows(['renter_occupied_housing_units_paying_cash_median_gross_rent',
                              'owner_occupied_housing_units_lower_value_quartile', 
                              'median_year_structure_built', 'owner_occupied_housing_units_median_value',
                              'owner_occupied_housing_units_upper_value_quartile'], ACS)

Rows with missingness in renter_occupied_housing_units_paying_cash_median_gross_rent: 0
Cleaning complete.


Rows with missingness in owner_occupied_housing_units_lower_value_quartile: 0
Cleaning complete.


Rows with missingness in median_year_structure_built: 0
Cleaning complete.


Rows with missingness in owner_occupied_housing_units_median_value: 0
Cleaning complete.


Rows with missingness in owner_occupied_housing_units_upper_value_quartile: 0
Cleaning complete.




##### Taking a closer look into the high missingness columns<br>
##### Marital Status

In [28]:
ACS.loc[ACS['pop_now_married'].isna()]

,geo_id,total_pop,households,male_pop,female_pop,median_age,male_under_5,male_5_to_9,male_10_to_14,male_15_to_17,male_18_to_19,male_20,male_21,male_22_to_24,male_25_to_29,male_30_to_34,male_35_to_39,male_40_to_44,male_45_to_49,male_50_to_54,male_55_to_59,male_65_to_66,male_67_to_69,male_70_to_74,male_75_to_79,male_80_to_84,male_85_and_over,female_under_5,female_5_to_9,female_10_to_14,female_15_to_17,female_18_to_19,female_20,female_21,female_22_to_24,female_25_to_29,female_30_to_34,female_35_to_39,female_40_to_44,female_45_to_49,female_50_to_54,female_55_to_59,female_60_to_61,female_62_to_64,female_65_to_66,female_67_to_69,female_70_to_74,female_75_to_79,female_80_to_84,female_85_and_over,white_pop,population_1_year_and_over,population_3_years_over,pop_5_years_over,pop_15_and_over,pop_16_over,pop_25_years_over,pop_25_64,pop_never_married,pop_now_married,pop_separated,pop_widowed,pop_divorced,not_us_citizen_pop,black_pop,asian_pop,hispanic_pop,amerindian_pop,other_race_pop,two_or_more_races_pop,hispanic_any_race,not_hispanic_pop,asian_male_45_54,asian_male_55_64,black_male_45_54,black_male_55_64,hispanic_male_45_54,hispanic_male_55_64,white_male_45_54,white_male_55_64,pop_determined_poverty_status,poverty,housing_units,renter_occupied_housing_units_paying_cash_median_gross_rent,owner_occupied_housing_units_lower_value_quartile,owner_occupied_housing_units_median_value,owner_occupied_housing_units_upper_value_quartile,occupied_housing_units,housing_units_renter_occupied,vacant_housing_units,vacant_housing_units_for_rent,vacant_housing_units_for_sale,dwellings_1_units_detached,dwellings_1_units_attached,dwellings_2_units,dwellings_3_to_4_units,dwellings_5_to_9_units,dwellings_10_to_19_units,dwellings_20_to_49_units,dwellings_50_or_more_units,mobile_homes,housing_built_2005_or_later,housing_built_2000_to_2004,housing_built_1939_or_earlier,median_year_structure_built,married_households,nonfamily_households,family_households,households_public_asst_or_food_stamps,male_male_households,female_female_households,children,children_in_single_female_hh,median_rent,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,rent_35_to_40_percent,rent_30_to_35_percent,rent_25_to_30_percent,rent_20_to_25_percent,rent_15_to_20_percent,rent_10_to_15_percent,rent_under_10_percent,owner_occupied_housing_units,million_dollar_housing_units,mortgaged_housing_units,different_house_year_ago_different_city,different_house_year_ago_same_city,families_with_young_children,two_parent_families_with_young_children,two_parents_in_labor_force_families_with_young_children,two_parents_father_in_labor_force_families_with_young_children,two_parents_mother_in_labor_force_families_with_young_children,two_parents_not_in_labor_force_families_with_young_children,one_parent_families_with_young_children,father_one_parent_families_with_young_children,father_in_labor_force_one_parent_families_with_young_children,commute_less_10_mins,commute_10_14_mins,commute_15_19_mins,commute_20_24_mins,commute_25_29_mins,commute_30_34_mins,commute_35_44_mins,commute_60_more_mins,commute_45_59_mins,commuters_16_over,walked_to_work,worked_at_home,no_car,no_cars,one_car,two_cars,three_cars,four_more_cars,aggregate_travel_time_to_work,commuters_by_public_transportation,commuters_by_bus,commuters_by_car_truck_van,commuters_by_carpool,commuters_by_subway_or_elevated,commuters_drove_alone,group_quarters,associates_degree,bachelors_degree,high_school_diploma,less_one_year_college,masters_degree,one_year_more_college,less_than_high_school_graduate,high_school_including_ged,bachelors_degree_2,bachelors_degree_or_higher_25_64,graduate_professional_degree,some_college_and_associates_degree,male_45_64_associates_degree,male_45_64_bachelors_degree,male_45_64_graduate_degree,male_45_64_less_than_9_grade,male_45_64_grade_9_12,male_45_64_high_school,male_45_64_some_college,male_45_to_64,employed_pop,unemployed_pop,pop_in_labor_force,not_in_labor_force,workers_16_and_over,armed_forces,civilian_lab

In [29]:
ACS.loc[~ACS['pop_now_married'].isna()]['year'].unique()

array([2014])

Based on the results above, it seems like the data pertaining to marital status was either not reported by the individuals completing the survey or was not collected by the survey department, with the latter being the most probable example. With an 80% reported missingness, with no way of comfortably imputing, the columns pertaining to marital status will be dropped.

In [30]:
#Dropping marital status columns
ACS.drop(columns = ['pop_never_married', 'pop_now_married', 'pop_separated', 'pop_widowed', 'pop_divorced'], 
         inplace = True)

##### Languages Spoken at Home

In [31]:
ACS.loc[~ACS['speak_spanish_at_home'].isna()]['year'].unique()

array([2015, 2014])

In [32]:
ACS.loc[~ACS['speak_only_english_at_home'].isna()]['year'].unique()

array([2015, 2014])

In [33]:
ACS.loc[~ACS['speak_only_english_at_home'].isna()]['year'].unique()

array([2015, 2014])

In [34]:
ACS.loc[~ACS['speak_spanish_at_home_low_english'].isna()]['year'].unique()

array([2015, 2014])

Based on the above results, it can be concluded that the only in 2014 and 2015 data about languages spoken at home was collected. With a 60% reported missingness, with no way of comfortably imputing, the columns pertaining to languages spoken at home will be dropped.

In [35]:
ACS.drop(columns = ['speak_spanish_at_home', 'speak_only_english_at_home', 'speak_only_english_at_home',
                   'speak_spanish_at_home_low_english'], inplace = True)

### Population Age Groups

To further reduce the number of columns in the ACS data set, columns reporting various population groups will be further grouped, with the total values removed from the data set.<br><br>
Before doing so, total values will be checked with the "dummified" groupings.

#### Checking Male Population Grouping and Totals

Age groups will further be combined into the following:<br>
- Males under 18
- Males from 18 to 24
- Males from 25 to 49
- Males from 50 to 66
- Males above 67

In [36]:
#Combining age groups
ACS['male_under_18'] = ACS['male_under_5'] + ACS['male_5_to_9'] + ACS['male_10_to_14'] + ACS['male_15_to_17']
ACS['male_18_to_24'] = ACS['male_18_to_19'] + ACS['male_20'] + ACS['male_21'] + ACS['male_22_to_24']
ACS['male_25_to_49'] = ACS['male_25_to_29'] + ACS['male_30_to_34'] + ACS['male_35_to_39'] + ACS['male_40_to_44'] +\
                        ACS['male_45_to_49']
ACS['male_50_to_66'] = ACS['male_50_to_54'] + ACS['male_55_to_59'] + ACS['male_65_to_66']
ACS['male_above_67'] = ACS['male_67_to_69'] + ACS['male_70_to_74'] + ACS['male_75_to_79'] + ACS['male_80_to_84'] +\
                        ACS['male_85_and_over']

In [37]:
def clean_group_distribution(df, col_start, check_on, bound_low = None, bound_high = None, new_group = None):
    """
    This function cleans up "dummified" values of a set of columns by checking whether the 
    aggregate of the columns in question add up to the total in the original dataframe. Should 
    it not add up, a new column is generated with the difference between the sum and total.
    
    Arguments
    
    df         : (Pandas dataframe) dataframe of interest
    col_start  : (str) string indicating what each dummified column begins with - must be in quotations! 
    check_on   : (Pandas dataframe col) column to check against the dummified columns
    bound_low  : (int) value at which dummified column list begins (default = None)
    bound_high : (str) name of first column after dummified columns of interest (default = None) - 
                       must be in quotations!
    new_group  : (int) If 1 is inputted, the function assumes a new grouping has been introduced for the 
                       dummified columns in question (default = None)
    
    Returns
    
    df         : (Pandas dataframe) cleaned version of inputted dataframe
    
    """
    check_on = check_on.astype(float)
    
    #Creating a list of columns of interest
    cols_interest = [x for x in df.columns if x.startswith(col_start)]
    
    #Additional cleaning if required (specified by bounds low and high)
    if bound_low:
        cols_interest = cols_interest[bound_low:cols_interest.index(bound_high)]
    
    #Creating a check column
    df[cols_interest] = df[cols_interest].astype(float)
    df['check'] = df[cols_interest].sum(axis = 1)
    df['check'] = df['check'].astype(float)
    
    #Comparing total of check_on with check values
    print(f'Total rows in requested dataframe: {df.shape[0]}')
    print(f'Total rows with unmatched employed population: {df.loc[check_on != df.check].shape[0]}\n\n')
    
    #Creating the remainder of specified group as column
    df['x'] = abs(check_on - df['check'])
    df['x'] = df['x'].astype(float)
    
    #Reporting percentage of values that haven't been appropriately reported
    perc_x = round(df['x'].sum() / check_on.sum() * 100, 2)
    print(f'Total percentage of values that haven\'t been appropriately reported: {perc_x}%')
    
    #Dropping intermediary operation columns
    df.drop(columns = ['check', check_on.name], inplace = True)
    
    #Dropping splitting columns
    if new_group == 1:
        df.drop(columns = cols_interest, inplace = True)
    
    if perc_x == 0:
        df.drop(columns = 'x', inplace = True)
        print('Creating difference column in not necessary.')
        
    #Renaming remainder column
    df.rename(columns = {'x': col_start+'x'}, inplace = True)
    
    return df.head(3)

In [38]:
clean_group_distribution(ACS, 'male_', ACS['male_pop'], 1, 'male_male_households', 1)

Total rows in requested dataframe: 11079
Total rows with unmatched employed population: 11079


Total percentage of values that haven't been appropriately reported: 5.24%


,geo_id,total_pop,households,female_pop,median_age,female_under_5,female_5_to_9,female_10_to_14,female_15_to_17,female_18_to_19,female_20,female_21,female_22_to_24,female_25_to_29,female_30_to_34,female_35_to_39,female_40_to_44,female_45_to_49,female_50_to_54,female_55_to_59,female_60_to_61,female_62_to_64,female_65_to_66,female_67_to_69,female_70_to_74,female_75_to_79,female_80_to_84,female_85_and_over,white_pop,population_1_year_and_over,population_3_years_over,pop_5_years_over,pop_15_and_over,pop_16_over,pop_25_years_over,pop_25_64,not_us_citizen_pop,black_pop,asian_pop,hispanic_pop,amerindian_pop,other_race_pop,two_or_more_races_pop,hispanic_any_race,not_hispanic_pop,asian_male_45_54,asian_male_55_64,black_male_45_54,black_male_55_64,hispanic_male_45_54,hispanic_male_55_64,white_male_45_54,white_male_55_64,pop_determined_poverty_status,poverty,housing_units,renter_occupied_housing_units_paying_cash_median_gross_rent,owner_occupied_housing_units_lower_value_quartile,owner_occupied_housing_units_median_value,owner_occupied_housing_units_upper_value_quartile,occupied_housing_units,housing_units_renter_occupied,vacant_housing_units,vacant_housing_units_for_rent,vacant_housing_units_for_sale,dwellings_1_units_detached,dwellings_1_units_attached,dwellings_2_units,dwellings_3_to_4_units,dwellings_5_to_9_units,dwellings_10_to_19_units,dwellings_20_to_49_units,dwellings_50_or_more_units,mobile_homes,housing_built_2005_or_later,housing_built_2000_to_2004,housing_built_1939_or_earlier,median_year_structure_built,married_households,nonfamily_households,family_households,households_public_asst_or_food_stamps,male_male_households,female_female_households,children,children_in_single_female_hh,median_rent,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,rent_35_to_40_percent,rent_30_to_35_percent,rent_25_to_30_percent,rent_20_to_25_percent,rent_15_to_20_percent,rent_10_to_15_percent,rent_under_10_percent,owner_occupied_housing_units,million_dollar_housing_units,mortgaged_housing_units,different_house_year_ago_different_city,different_house_year_ago_same_city,families_with_young_children,two_parent_families_with_young_children,two_parents_in_labor_force_families_with_young_children,two_parents_father_in_labor_force_families_with_young_children,two_parents_mother_in_labor_force_families_with_young_children,two_parents_not_in_labor_force_families_with_young_children,one_parent_families_with_young_children,father_one_parent_families_with_young_children,father_in_labor_force_one_parent_families_with_young_children,commute_less_10_mins,commute_10_14_mins,commute_15_19_mins,commute_20_24_mins,commute_25_29_mins,commute_30_34_mins,commute_35_44_mins,commute_60_more_mins,commute_45_59_mins,commuters_16_over,walked_to_work,worked_at_home,no_car,no_cars,one_car,two_cars,three_cars,four_more_cars,aggregate_travel_time_to_work,commuters_by_public_transportation,commuters_by_bus,commuters_by_car_truck_van,commuters_by_carpool,commuters_by_subway_or_elevated,commuters_drove_alone,group_quarters,associates_degree,bachelors_degree,high_school_diploma,less_one_year_college,masters_degree,one_year_more_college,less_than_high_school_graduate,high_school_including_ged,bachelors_degree_2,bachelors_degree_or_higher_25_64,graduate_professional_degree,some_college_and_associates_degree,male_45_64_associates_degree,male_45_64_bachelors_degree,male_45_64_graduate_degree,male_45_64_less_than_9_grade,male_45_64_grade_9_12,male_45_64_high_school,male_45_64_some_college,male_45_to_64,employed_pop,unemployed_pop,pop_in_labor_force,not_in_labor_force,workers_16_and_over,armed_forces,civilian_labor_force,employed_agriculture_forestry_fishing_hunting_mining,employed_arts_entertainment_recreation_accommodation_food,employed_construction,employed_education_health_social,employed_finance_insurance_real_estate,employed_information,employed_manufacturing,employed_other_services_not_public_admin,employed_public_administration,employed_retail_trade,employed_sci

#### Checking Female Population Grouping and Totals

Age groups will further be combined into the following:<br>
- Females under 18
- Females from 18 to 24
- Females from 25 to 49
- Females from 50 to 66
- Females above 67

In [39]:
#Combining age groups
ACS['female_under_18'] = ACS['female_under_5'] + ACS['female_5_to_9'] + ACS['female_10_to_14'] + ACS['female_15_to_17']
ACS['female_18_to_24'] = ACS['female_18_to_19'] + ACS['female_20'] + ACS['female_21'] + ACS['female_22_to_24']
ACS['female_25_to_49'] = ACS['female_25_to_29'] + ACS['female_30_to_34'] + ACS['female_35_to_39'] + ACS['female_40_to_44'] +\
                        ACS['female_45_to_49']
ACS['female_50_to_66'] = ACS['female_50_to_54'] + ACS['female_55_to_59'] + ACS['female_65_to_66']
ACS['female_above_67'] = ACS['female_67_to_69'] + ACS['female_70_to_74'] + ACS['female_75_to_79'] + ACS['female_80_to_84'] +\
                        ACS['female_85_and_over']

In [40]:
clean_group_distribution(ACS, 'female_', ACS['female_pop'], 1, 'female_female_households', 1)

Total rows in requested dataframe: 11079
Total rows with unmatched employed population: 0


Total percentage of values that haven't been appropriately reported: 0.0%
Creating difference column in not necessary.


,geo_id,total_pop,households,median_age,white_pop,population_1_year_and_over,population_3_years_over,pop_5_years_over,pop_15_and_over,pop_16_over,pop_25_years_over,pop_25_64,not_us_citizen_pop,black_pop,asian_pop,hispanic_pop,amerindian_pop,other_race_pop,two_or_more_races_pop,hispanic_any_race,not_hispanic_pop,asian_male_45_54,asian_male_55_64,black_male_45_54,black_male_55_64,hispanic_male_45_54,hispanic_male_55_64,white_male_45_54,white_male_55_64,pop_determined_poverty_status,poverty,housing_units,renter_occupied_housing_units_paying_cash_median_gross_rent,owner_occupied_housing_units_lower_value_quartile,owner_occupied_housing_units_median_value,owner_occupied_housing_units_upper_value_quartile,occupied_housing_units,housing_units_renter_occupied,vacant_housing_units,vacant_housing_units_for_rent,vacant_housing_units_for_sale,dwellings_1_units_detached,dwellings_1_units_attached,dwellings_2_units,dwellings_3_to_4_units,dwellings_5_to_9_units,dwellings_10_to_19_units,dwellings_20_to_49_units,dwellings_50_or_more_units,mobile_homes,housing_built_2005_or_later,housing_built_2000_to_2004,housing_built_1939_or_earlier,median_year_structure_built,married_households,nonfamily_households,family_households,households_public_asst_or_food_stamps,male_male_households,female_female_households,children,children_in_single_female_hh,median_rent,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,rent_35_to_40_percent,rent_30_to_35_percent,rent_25_to_30_percent,rent_20_to_25_percent,rent_15_to_20_percent,rent_10_to_15_percent,rent_under_10_percent,owner_occupied_housing_units,million_dollar_housing_units,mortgaged_housing_units,different_house_year_ago_different_city,different_house_year_ago_same_city,families_with_young_children,two_parent_families_with_young_children,two_parents_in_labor_force_families_with_young_children,two_parents_father_in_labor_force_families_with_young_children,two_parents_mother_in_labor_force_families_with_young_children,two_parents_not_in_labor_force_families_with_young_children,one_parent_families_with_young_children,father_one_parent_families_with_young_children,father_in_labor_force_one_parent_families_with_young_children,commute_less_10_mins,commute_10_14_mins,commute_15_19_mins,commute_20_24_mins,commute_25_29_mins,commute_30_34_mins,commute_35_44_mins,commute_60_more_mins,commute_45_59_mins,commuters_16_over,walked_to_work,worked_at_home,no_car,no_cars,one_car,two_cars,three_cars,four_more_cars,aggregate_travel_time_to_work,commuters_by_public_transportation,commuters_by_bus,commuters_by_car_truck_van,commuters_by_carpool,commuters_by_subway_or_elevated,commuters_drove_alone,group_quarters,associates_degree,bachelors_degree,high_school_diploma,less_one_year_college,masters_degree,one_year_more_college,less_than_high_school_graduate,high_school_including_ged,bachelors_degree_2,bachelors_degree_or_higher_25_64,graduate_professional_degree,some_college_and_associates_degree,male_45_64_associates_degree,male_45_64_bachelors_degree,male_45_64_graduate_degree,male_45_64_less_than_9_grade,male_45_64_grade_9_12,male_45_64_high_school,male_45_64_some_college,male_45_to_64,employed_pop,unemployed_pop,pop_in_labor_force,not_in_labor_force,workers_16_and_over,armed_forces,civilian_labor_force,employed_agriculture_forestry_fishing_hunting_mining,employed_arts_entertainment_recreation_accommodation_food,employed_construction,employed_education_health_social,employed_finance_insurance_real_estate,employed_information,employed_manufacturing,employed_other_services_not_public_admin,employed_public_administration,employed_retail_trade,employed_science_management_admin_waste,employed_transportation_warehousing_utilities,employed_wholesale_trade,occupation_management_arts,occupation_natural_resources_construction_maintenance,occupation_production_transportation_material,occupation_sales_office,occupation_services,management_business_sci_arts_employed,sales_office_employed,in_grades_1_to_4,in_grades_5_to_8,in_

Seems like women always have a reported age! Very different from the male population.

#### Population Over Certain Age Grouping

There are columns indicating the total number of people over a certain age group. As age grouping has been performed prior, these columns will be dropped along with median age (this can be calculated outside of modelling).

In [41]:
ACS.drop(columns = ['median_age', 'population_1_year_and_over', 'population_3_years_over', 
                          'pop_5_years_over', 'pop_15_and_over', 'pop_16_over', 'pop_25_years_over', 
                          'pop_25_64'], inplace = True)

Additionally, since all children are defined as people less than 18 years old, this has already been encompassed in the male / female population split. As such, the *children* column will be dropped. 

In [42]:
#Additional check that all males and females under 18 are the same as the children column
children_df = ACS[['male_under_18', 'female_under_18']].copy()
children_df['sum'] = children_df['male_under_18'] + children_df['female_under_18']
children_df['children'] = ACS['children'].copy()
children_df.loc[children_df['sum'] != children_df['children']]

,male_under_18,female_under_18,sum,children


In [43]:
#Dropping the children column
ACS.drop(columns = 'children', inplace = True)

### Employment Columns

Both *employed_* and *occupation_* type columns cover the same material - which sector a working individual is working. To have a general understanding of occupation, all *employed_* columns will be dropped.

In [44]:
ACS = ACS[ACS.columns.drop(list(ACS.filter(regex = 'employed')))]

Additionally, all columns relating to total employment, such as *civilian_labor_force* among others, will be dropped. Before doing so, a check will be performed on all occupations to ensure no employed worker is missed.

In [45]:
clean_group_distribution(ACS, 'occupation_', ACS['pop_in_labor_force'])

Total rows in requested dataframe: 11079
Total rows with unmatched employed population: 11070


Total percentage of values that haven't been appropriately reported: 7.93%


,geo_id,total_pop,households,white_pop,not_us_citizen_pop,black_pop,asian_pop,hispanic_pop,amerindian_pop,other_race_pop,two_or_more_races_pop,hispanic_any_race,not_hispanic_pop,asian_male_45_54,asian_male_55_64,black_male_45_54,black_male_55_64,hispanic_male_45_54,hispanic_male_55_64,white_male_45_54,white_male_55_64,pop_determined_poverty_status,poverty,housing_units,renter_occupied_housing_units_paying_cash_median_gross_rent,owner_occupied_housing_units_lower_value_quartile,owner_occupied_housing_units_median_value,owner_occupied_housing_units_upper_value_quartile,occupied_housing_units,housing_units_renter_occupied,vacant_housing_units,vacant_housing_units_for_rent,vacant_housing_units_for_sale,dwellings_1_units_detached,dwellings_1_units_attached,dwellings_2_units,dwellings_3_to_4_units,dwellings_5_to_9_units,dwellings_10_to_19_units,dwellings_20_to_49_units,dwellings_50_or_more_units,mobile_homes,housing_built_2005_or_later,housing_built_2000_to_2004,housing_built_1939_or_earlier,median_year_structure_built,married_households,nonfamily_households,family_households,households_public_asst_or_food_stamps,male_male_households,female_female_households,children_in_single_female_hh,median_rent,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,rent_35_to_40_percent,rent_30_to_35_percent,rent_25_to_30_percent,rent_20_to_25_percent,rent_15_to_20_percent,rent_10_to_15_percent,rent_under_10_percent,owner_occupied_housing_units,million_dollar_housing_units,mortgaged_housing_units,different_house_year_ago_different_city,different_house_year_ago_same_city,families_with_young_children,two_parent_families_with_young_children,two_parents_in_labor_force_families_with_young_children,two_parents_father_in_labor_force_families_with_young_children,two_parents_mother_in_labor_force_families_with_young_children,two_parents_not_in_labor_force_families_with_young_children,one_parent_families_with_young_children,father_one_parent_families_with_young_children,father_in_labor_force_one_parent_families_with_young_children,commute_less_10_mins,commute_10_14_mins,commute_15_19_mins,commute_20_24_mins,commute_25_29_mins,commute_30_34_mins,commute_35_44_mins,commute_60_more_mins,commute_45_59_mins,commuters_16_over,walked_to_work,worked_at_home,no_car,no_cars,one_car,two_cars,three_cars,four_more_cars,aggregate_travel_time_to_work,commuters_by_public_transportation,commuters_by_bus,commuters_by_car_truck_van,commuters_by_carpool,commuters_by_subway_or_elevated,commuters_drove_alone,group_quarters,associates_degree,bachelors_degree,high_school_diploma,less_one_year_college,masters_degree,one_year_more_college,less_than_high_school_graduate,high_school_including_ged,bachelors_degree_2,bachelors_degree_or_higher_25_64,graduate_professional_degree,some_college_and_associates_degree,male_45_64_associates_degree,male_45_64_bachelors_degree,male_45_64_graduate_degree,male_45_64_less_than_9_grade,male_45_64_grade_9_12,male_45_64_high_school,male_45_64_some_college,male_45_to_64,not_in_labor_force,workers_16_and_over,armed_forces,civilian_labor_force,occupation_management_arts,occupation_natural_resources_construction_maintenance,occupation_production_transportation_material,occupation_sales_office,occupation_services,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_school,in_undergrad_college,year,male_under_18,male_18_to_24,male_25_to_49,male_50_to_66,male_above_67,male_x,female_under_18,female_18_to_24,female_25_to_49,female_50_to_66,female_above_67,occupation_x
0,11959,784,327,697,0,18,30,33,0,0,6,33,751,11,0,0,1,0,0,96,40,784,66,1571,1630,776000,1.0469e+06,2e+06,327,76,1244,0,0,1535,0,18,0,18,0,0,0,0,0,30,40,1975,212,82,245,10,0,0,29,1400,0,19,0,21,0,0,0,22,11,3,251,56,125,9,0,55,39,39,0,0,0,16,6,6,34,33,33,40,8,16,8,42,50,264,0,66,10,10,62,174,38,43,8341,11,2,249,13,0,236,9,39,174,60,25,161,47,10,63,174,269,235,111,11,56,57,5,2,10,7,148,292,330,0,336,179.0,15.0,23.0,106.0,13.0,0,70,34,194,32,2018,65,24,71,109,71,23.0,102,0,87,116,86,0

In [46]:
#Dropping all other columns
ACS = ACS.drop(columns = ['civilian_labor_force', 'workers_16_and_over'])

### Commuters

For the columns looking at the number of commuters, typically when people look at housing (rent or purchase), it is *time* to workplace / school / etc that plays a pivotal role. As such, all of the columns pertaining to the *type* of commuter will be dropped.

In [123]:
ACS = ACS[ACS.columns.drop(list(ACS.filter(regex = 'commuters_')))]
ACS.drop(columns = 'walked_to_work', inplace = True)

### People in Schools

For the columns looking at the number of people in school, the total column will be dropped. Before doing so, a check will be performed on all grade types to ensure no student is missed.

In [48]:
clean_group_distribution(ACS, 'in_grades_', ACS['in_school'])

Total rows in requested dataframe: 11079
Total rows with unmatched employed population: 11079


Total percentage of values that haven't been appropriately reported: 39.38%


,geo_id,total_pop,households,white_pop,not_us_citizen_pop,black_pop,asian_pop,hispanic_pop,amerindian_pop,other_race_pop,two_or_more_races_pop,hispanic_any_race,not_hispanic_pop,asian_male_45_54,asian_male_55_64,black_male_45_54,black_male_55_64,hispanic_male_45_54,hispanic_male_55_64,white_male_45_54,white_male_55_64,pop_determined_poverty_status,poverty,housing_units,renter_occupied_housing_units_paying_cash_median_gross_rent,owner_occupied_housing_units_lower_value_quartile,owner_occupied_housing_units_median_value,owner_occupied_housing_units_upper_value_quartile,occupied_housing_units,housing_units_renter_occupied,vacant_housing_units,vacant_housing_units_for_rent,vacant_housing_units_for_sale,dwellings_1_units_detached,dwellings_1_units_attached,dwellings_2_units,dwellings_3_to_4_units,dwellings_5_to_9_units,dwellings_10_to_19_units,dwellings_20_to_49_units,dwellings_50_or_more_units,mobile_homes,housing_built_2005_or_later,housing_built_2000_to_2004,housing_built_1939_or_earlier,median_year_structure_built,married_households,nonfamily_households,family_households,households_public_asst_or_food_stamps,male_male_households,female_female_households,children_in_single_female_hh,median_rent,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,rent_35_to_40_percent,rent_30_to_35_percent,rent_25_to_30_percent,rent_20_to_25_percent,rent_15_to_20_percent,rent_10_to_15_percent,rent_under_10_percent,owner_occupied_housing_units,million_dollar_housing_units,mortgaged_housing_units,different_house_year_ago_different_city,different_house_year_ago_same_city,families_with_young_children,two_parent_families_with_young_children,two_parents_in_labor_force_families_with_young_children,two_parents_father_in_labor_force_families_with_young_children,two_parents_mother_in_labor_force_families_with_young_children,two_parents_not_in_labor_force_families_with_young_children,one_parent_families_with_young_children,father_one_parent_families_with_young_children,father_in_labor_force_one_parent_families_with_young_children,commute_less_10_mins,commute_10_14_mins,commute_15_19_mins,commute_20_24_mins,commute_25_29_mins,commute_30_34_mins,commute_35_44_mins,commute_60_more_mins,commute_45_59_mins,walked_to_work,worked_at_home,no_car,no_cars,one_car,two_cars,three_cars,four_more_cars,aggregate_travel_time_to_work,commuters_by_public_transportation,commuters_by_bus,commuters_by_car_truck_van,commuters_by_carpool,commuters_by_subway_or_elevated,commuters_drove_alone,group_quarters,associates_degree,bachelors_degree,high_school_diploma,less_one_year_college,masters_degree,one_year_more_college,less_than_high_school_graduate,high_school_including_ged,bachelors_degree_2,bachelors_degree_or_higher_25_64,graduate_professional_degree,some_college_and_associates_degree,male_45_64_associates_degree,male_45_64_bachelors_degree,male_45_64_graduate_degree,male_45_64_less_than_9_grade,male_45_64_grade_9_12,male_45_64_high_school,male_45_64_some_college,male_45_to_64,not_in_labor_force,armed_forces,occupation_management_arts,occupation_natural_resources_construction_maintenance,occupation_production_transportation_material,occupation_sales_office,occupation_services,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_undergrad_college,year,male_under_18,male_18_to_24,male_25_to_49,male_50_to_66,male_above_67,male_x,female_under_18,female_18_to_24,female_25_to_49,female_50_to_66,female_above_67,occupation_x,commuters_x,in_grades_x
0,11959,784,327,697,0,18,30,33,0,0,6,33,751,11,0,0,1,0,0,96,40,784,66,1571,1630,776000,1.0469e+06,2e+06,327,76,1244,0,0,1535,0,18,0,18,0,0,0,0,0,30,40,1975,212,82,245,10,0,0,29,1400,0,19,0,21,0,0,0,22,11,3,251,56,125,9,0,55,39,39,0,0,0,16,6,6,34,33,33,40,8,16,8,42,50,0,66,10,10,62,174,38,43,8341,11.0,2.0,249.0,13.0,0.0,236,9,39,174,60,25,161,47,10,63,174,269,235,111,11,56,57,5,2,10,7,148,292,0,179.0,15.0,23.0,106.0,13.0,0.0,70.0,34.0,32,2018,65,24,71,109,71,23.0,102,0,87,116,86,0.0,11.0,90.0
1,48320,4839,2406,4264,111,201,4

### Missingness and Shape Final Check

In [49]:
missingness(ACS)

{}

In [50]:
ACS.shape

(11079, 149)

## Further Cleaning to Reduce Features

### Income Spent on Rent

Rent and income related data will be coming from ZRI and IRS data respectively. As such, columns relating to how much income is spent on rent will be dropped.

In [61]:
ACS.drop(columns = ['median_rent', 'rent_burden_not_computed', 'rent_over_50_percent', 'rent_40_to_50_percent',
                    'rent_35_to_40_percent', 'rent_30_to_35_percent', 'rent_25_to_30_percent',
                    'rent_20_to_25_percent', 'rent_15_to_20_percent', 'rent_10_to_15_percent',
                    'rent_under_10_percent'], inplace = True)

### Housing Units

In [63]:
clean_group_distribution(ACS, 'vacant_housing_units_', ACS['vacant_housing_units'])

Total rows in requested dataframe: 11079
Total rows with unmatched employed population: 11035


Total percentage of values that haven't been appropriately reported: 62.52%


,geo_id,total_pop,households,white_pop,not_us_citizen_pop,black_pop,asian_pop,hispanic_pop,amerindian_pop,other_race_pop,two_or_more_races_pop,hispanic_any_race,not_hispanic_pop,asian_male_45_54,asian_male_55_64,black_male_45_54,black_male_55_64,hispanic_male_45_54,hispanic_male_55_64,white_male_45_54,white_male_55_64,pop_determined_poverty_status,poverty,housing_units,renter_occupied_housing_units_paying_cash_median_gross_rent,owner_occupied_housing_units_lower_value_quartile,owner_occupied_housing_units_median_value,owner_occupied_housing_units_upper_value_quartile,occupied_housing_units,housing_units_renter_occupied,vacant_housing_units_for_rent,vacant_housing_units_for_sale,dwellings_1_units_detached,dwellings_1_units_attached,dwellings_2_units,dwellings_3_to_4_units,dwellings_5_to_9_units,dwellings_10_to_19_units,dwellings_20_to_49_units,dwellings_50_or_more_units,mobile_homes,housing_built_2005_or_later,housing_built_2000_to_2004,housing_built_1939_or_earlier,median_year_structure_built,married_households,nonfamily_households,family_households,households_public_asst_or_food_stamps,male_male_households,female_female_households,children_in_single_female_hh,owner_occupied_housing_units,million_dollar_housing_units,mortgaged_housing_units,different_house_year_ago_different_city,different_house_year_ago_same_city,families_with_young_children,two_parent_families_with_young_children,two_parents_in_labor_force_families_with_young_children,two_parents_father_in_labor_force_families_with_young_children,two_parents_mother_in_labor_force_families_with_young_children,two_parents_not_in_labor_force_families_with_young_children,one_parent_families_with_young_children,father_one_parent_families_with_young_children,father_in_labor_force_one_parent_families_with_young_children,commute_less_10_mins,commute_10_14_mins,commute_15_19_mins,commute_20_24_mins,commute_25_29_mins,commute_30_34_mins,commute_35_44_mins,commute_60_more_mins,commute_45_59_mins,walked_to_work,worked_at_home,no_car,no_cars,one_car,two_cars,three_cars,four_more_cars,aggregate_travel_time_to_work,commuters_by_public_transportation,commuters_by_bus,commuters_by_car_truck_van,commuters_by_carpool,commuters_by_subway_or_elevated,commuters_drove_alone,group_quarters,associates_degree,bachelors_degree,high_school_diploma,less_one_year_college,masters_degree,one_year_more_college,less_than_high_school_graduate,high_school_including_ged,bachelors_degree_2,bachelors_degree_or_higher_25_64,graduate_professional_degree,some_college_and_associates_degree,male_45_64_associates_degree,male_45_64_bachelors_degree,male_45_64_graduate_degree,male_45_64_less_than_9_grade,male_45_64_grade_9_12,male_45_64_high_school,male_45_64_some_college,male_45_to_64,not_in_labor_force,armed_forces,occupation_management_arts,occupation_natural_resources_construction_maintenance,occupation_production_transportation_material,occupation_sales_office,occupation_services,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_undergrad_college,year,male_under_18,male_18_to_24,male_25_to_49,male_50_to_66,male_above_67,male_x,female_under_18,female_18_to_24,female_25_to_49,female_50_to_66,female_above_67,occupation_x,commuters_x,in_grades_x,vacant_housing_units_x
0,11959,784,327,697,0,18,30,33,0,0,6,33,751,11,0,0,1,0,0,96,40,784,66,1571,1630,776000,1.0469e+06,2e+06,327,76,0.0,0.0,1535,0,18,0,18,0,0,0,0,0,30,40,1975,212,82,245,10,0,0,29,251,56,125,9,0,55,39,39,0,0,0,16,6,6,34,33,33,40,8,16,8,42,50,0,66,10,10,62,174,38,43,8341,11.0,2.0,249.0,13.0,0.0,236,9,39,174,60,25,161,47,10,63,174,269,235,111,11,56,57,5,2,10,7,148,292,0,179.0,15.0,23.0,106.0,13.0,0.0,70.0,34.0,32,2018,65,24,71,109,71,23.0,102,0,87,116,86,0.0,11.0,90.0,1244.0
1,48320,4839,2406,4264,111,201,44,295,5,10,20,295,4544,0,0,18,0,0,5,295,346,4828,559,2591,935,102100,175500,269500,2406,767,26.0,44.0,2004,44,15,148,55,191,16,54,64,73,0,178,1959,855,1216,1190,138,6,0,223,1639,16,1159,578,99,264,217,163,54,0,0,47,0,0,196,400,283,385,275,426,175,205,285,

In [71]:
#Dropping the 'dummified' version of owner occupied housing units
ACS.drop(columns = ['owner_occupied_housing_units_lower_value_quartile',
                   'owner_occupied_housing_units_upper_value_quartile'], inplace = True)

The column *housing_units_renter_occupied* covers more renter occupied homes than the values shown in column *renter_occupied_housing_units_paying_cash_median_gross_rent*. As such, the latter column will be dropped. 

In [75]:
ACS.drop(columns = 'renter_occupied_housing_units_paying_cash_median_gross_rent', inplace = True)

In [119]:
ACS.drop(columns = ['occupied_housing_units'], inplace = True)

Median structure built will indicate a general idea of house age, as such housing built in specific years will be dropped.

In [81]:
ACS = ACS[ACS.columns.drop(list(ACS.filter(regex = 'housing_built_')))]

### Men in Ages 45-65

There were a significant number of features focusing on men ranging from 45 to 65 years of age. Since this data is encompassed in the general male population, these features will be dropped.

In [68]:
ACS = ACS[ACS.columns.drop(list(ACS.filter(regex = 'male_45_64')))]
ACS = ACS[ACS.columns.drop(list(ACS.filter(regex = 'male_45_54')))]
ACS = ACS[ACS.columns.drop(list(ACS.filter(regex = 'male_55_64')))]

In [104]:
ACS.drop(columns = 'male_45_to_64', inplace = True)

### Poverty

In [85]:
ACS[['pop_determined_poverty_status', 'poverty']].head(3)

,pop_determined_poverty_status,poverty
0,784,66
1,4828,559
2,5853,362


Looking at the two poverty columns, as well as the schema for ACS, these can be combined to achieve total number of persons living in poverty.

In [107]:
#Summing the two poverty columns
ACS['poverty_total'] = ACS['pop_determined_poverty_status'] + ACS['poverty']

#Dropping the two and keeping the total column
ACS.drop(columns = ['pop_determined_poverty_status', 'poverty'], inplace = True)

In [92]:
ACS.drop(columns = 'households_public_asst_or_food_stamps', inplace = True)

### Families with Young Children

The total column will be kept for this, whereas the detailed breakdown will be removed.

In [100]:
ACS = ACS[ACS.columns.drop(list(ACS.filter(regex = 'with_young_children')))]
ACS.drop(columns = 'children_in_single_female_hh', inplace = True)

### Population by Race

In [113]:
ACS[['white_pop', 'not_us_citizen_pop', 'black_pop', 'asian_pop', 'hispanic_pop', 'amerindian_pop',
     'other_race_pop', 'two_or_more_races_pop', 'hispanic_any_race', 'not_hispanic_pop']].head(3)

,white_pop,not_us_citizen_pop,black_pop,asian_pop,hispanic_pop,amerindian_pop,other_race_pop,two_or_more_races_pop,hispanic_any_race,not_hispanic_pop
0,697,0,18,30,33,0,0,6,33,751
1,4264,111,201,44,295,5,10,20,295,4544
2,5175,49,100,269,104,1,0,241,104,5786


Based on the results above, columns *hispanic_pop* and *hispanic_any_race* are the same, and column *not_hispanic_pop* is the sum of all races that are not hispanic. As such, the two columns will be dropped.

In [115]:
ACS.drop(columns = ['hispanic_any_race', 'not_hispanic_pop'], inplace = True)

### Other Columns

In [97]:
ACS.drop(columns = ['group_quarters', 'million_dollar_housing_units', 'mortgaged_housing_units'], inplace = True)

#Dropping columns related to family types in households
ACS = ACS[ACS.columns.drop(list(ACS.filter(regex = 'households')))]

Both columns *no_car* and *no_cars* describe the same phenomena, although cars is focused on households without cars, rather than people. This value will be kept for the analysis.

In [106]:
ACS.drop(columns = 'no_cars', inplace = True)

## Exporting ACS Data

In [138]:
ACS.to_csv('ACS.csv')